# Data extraction of trips using Dask dataframe

# Purpose
As a first step the time series data will be divided into trips, as a data reduction. Energy consumption can be calculated for each trip together with other aggregated quantities such as mean values, standard deviations etc. This will be used to analyze how much trips differ from each other over the year.

But the file is larger than the memory can take so this solution uses a Dask dataframe instead.

# Methodology
* Loop over the dask dataframe partitions and number the trips, save to partquet in each loop.

# Setup

In [1]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../../../')
from src.visualization import visualize

sys.path.append('../../../src/models/pipelines/longterm/scripts/prepdata/trip_statistics')
import trip_statistics

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq
import dask.dataframe

## Parameters

In [2]:
name='tycho_short_parquet_id'

In [3]:
from dask.distributed import Client, progress, TimeoutError
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:52835 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 7.45 GiB


In [4]:
file_path = f'{name}.parquet'
df = dask.dataframe.read_parquet(file_path)

In [5]:
path = f'{name}_statistics.parquet'
df_statistics = trip_statistics.process(df=df, path=path)

In [6]:
df_statistics.head()

,latitude,longitude,heading,power_em_thruster_1,power_em_thruster_2,power_em_thruster_3,power_em_thruster_4,power_em_thruster_total,cog,sin_pm1,sin_pm2,sin_pm3,sin_pm4,cos_pm1,cos_pm2,cos_pm3,cos_pm4,power_heeling,sog,delta_1,delta_2,delta_3,delta_4,trip_no,trip_time
trip_no,,,,,,,,,,,,,,,,,,,,,,,,,
3.0,56.037121,12.657407,66.003030,384.353535,403.802571,356.265381,348.166208,1492.587695,72.778990,0.006538,0.041393,-0.123441,-0.077197,-0.711043,-0.712979,-0.758449,-0.755415,2.035813,4.428171,0.139468,0.170051,-0.223215,-0.189404,3.0,544.000918
5.0,56.037667,12.658731,66.705934,408.187609,436.029668,358.720768,348.050611,1550.988656,72.043185,0.009410,0.043116,-0.013878,0.039514,-0.657692,-0.658992,-0.696761,-0.695680,5.315009,4.270926,0.108890,0.160508,0.045390,0.016251,5.0,572.500000
10.0,56.035085,12.653924,79.800967,480.036907,510.279438,501.637083,473.816344,1965.769772,254.918330,0.055449,0.152862,0.099324,0.088388,0.771472,0.780358,0.713399,0.717055,1.521968,4.374468,-1.598657,2.316994,1.885320,1.212720,10.0,568.500000
15.0,56.036816,12.656966,66.760370,395.350926,417.489815,373.098148,371.552778,1557.491667,73.650981,-0.029059,0.000641,-0.110686,-0.053767,-0.740453,-0.743191,-0.761008,-0.758853,0.138889,4.503771,-0.176034,-0.133500,-0.202908,-0.273004,15.0,539.500000
19.0,56.037421,12.656943,67.862407,444.539815,476.281481,414.644444,405.754630,1741.220370,72.709019,-0.020574,0.010590,-0.239037,-0.178825,-0.638910,-0.642411,-0.762249,-0.757678,4.821296,4.524063,0.201561,0.248450,-0.360110,-0.301560,19.0,539.519444


In [7]:
client.close()